# Vamos aprender a trabalhar com PDF usando o Python

- Regra geral: PDF foi feito justamente para bloquear muita coisa, então não é fácil "brincar" com um pdf
- Mesmo assim, Python tem várias bibliotecas que vão nos ajudar, vamos focar em 2:
    - PyPDF2
    - Tabula
- Ler e extrair informações de um PDF a gente consegue fazer.
- Escrever e Editar, aí já é outra história

### Para os nossos exemplos, vamos avaliar o Release de Resultados do 3º e 4º Trimestre de 2020 da Magazine Luiza

#### 1º Objetivo: Queremos conseguir separar apenas o DRE do Release de Resultados (Página 14) para enviar para a Diretoria, como fazemos?
    - Separar as páginas de um pdf

In [9]:
from pathlib import Path
import PyPDF2 as pyf

CAMINHO_REFERENCIA = Path('./Python txt e PDF 10 - Dados')
for arquivo in CAMINHO_REFERENCIA.iterdir():
    if arquivo.is_file():
        arquivo_PDF = pyf.PdfReader(arquivo)
        for pos, page in enumerate(arquivo_PDF.pages, 1):
            novo_PDF = pyf.PdfWriter()
            novo_PDF.add_page(page)
            with open(CAMINHO_REFERENCIA.joinpath(f"paginas/{arquivo.name.removesuffix(arquivo.suffix)} - {pos}{arquivo.suffix}"), 'wb') as novo_arquivo:
                novo_PDF.write(novo_arquivo)


#### 2º Objetivo: Com o Release de Resultados já separado página por página, queremos incluir apenas as Páginas de Destaque (Página 1), DRE (Página 14) e Balanço (Página 16).
    - Juntar vários pdfs em 1

In [10]:
from pathlib import Path
import PyPDF2 as pyf

CAMINHO_REFERENCIA = Path('./Python txt e PDF 10 - Dados/paginas')
num_paginas = [1, 14, 16]
novo_PDF = pyf.PdfWriter()
nome_referencia = 'MGLU_ER_3T20_POR'
paginas = ''
for numero in num_paginas:
    arquivo_PDF = pyf.PdfReader(CAMINHO_REFERENCIA.joinpath(f'{nome_referencia} - {numero}.pdf'))
    novo_PDF.add_page(arquivo_PDF.pages[0])
    paginas += f'{numero}_'
if len(paginas) > 0:
    paginas = paginas[:-1]
with open(CAMINHO_REFERENCIA.joinpath(f"{nome_referencia} - {paginas}.pdf"), 'wb') as novo_arquivo:
                novo_PDF.write(novo_arquivo)

### Extra: Para adicionar todas as páginas de 2 pdfs

In [12]:
from pathlib import Path
import PyPDF2 as pyf

pdf_mesclado = pyf.PdfMerger()
CAMINHO_REFERENCIA = Path('./Python txt e PDF 10 - Dados')
for arquivo in CAMINHO_REFERENCIA.iterdir():
    if arquivo.is_file():
        pdf_mesclado.append(arquivo)
with open(CAMINHO_REFERENCIA.joinpath(f"Arquivo Mesclado.pdf"), 'wb') as novo_arquivo:
    pdf_mesclado.write(novo_arquivo)


# Funcionalidades que podem ser úteis:

- Inserir arquivo no meio do outro
- Quero colocar dentro do Resultado do 4T20 os destaques do 3T20 para poder comparar os 2 dentro do mesmo relatório

In [2]:
from pathlib import Path
import PyPDF2 as pyf

pdf_mesclado = pyf.PdfMerger()
CAMINHO_REFERENCIA = Path('./Python txt e PDF 10 - Dados')
pdf_mesclado.append(CAMINHO_REFERENCIA.joinpath('MGLU_ER_4T20_POR.pdf'))
pdf_mesclado.merge(1, CAMINHO_REFERENCIA.joinpath('paginas/MGLU_ER_3T20_POR - 1.pdf'))

with open(CAMINHO_REFERENCIA.joinpath(f"Relatorio 2 Trimestres.pdf"), 'wb') as novo_arquivo:
    pdf_mesclado.write(novo_arquivo)


- Rodar Página

In [2]:
from pathlib import Path
import PyPDF2 as pyf

CAMINHO_REFERENCIA = Path('./Python txt e PDF 10 - Dados')
arquivo_PDF = pyf.PdfReader(CAMINHO_REFERENCIA.joinpath(f"MGLU_ER_3T20_POR.pdf"))
novo_PDF = pyf.PdfWriter()
for page in arquivo_PDF.pages:
    page.rotate(90)
    novo_PDF.add_page(page)
with open(CAMINHO_REFERENCIA.joinpath(f"Paginas Rotacionadas.pdf"), 'wb') as novo_arquivo:
    novo_PDF.write(novo_arquivo)


# Trabalhando com Textos e Informações Dentro do PDF

#### 1º Objetivo: Quero identificar como foram as Despesas com Vendas da MGLU
    - Pegar texto da página e identificar onde está essa informação

In [24]:
from pathlib import Path
import PyPDF2 as pyf

texto_referencia = "| Despesas com Vendas"

CAMINHO_REFERENCIA = Path('./Python txt e PDF 10 - Dados')
arquivo_PDF = pyf.PdfReader(CAMINHO_REFERENCIA.joinpath(f"MGLU_ER_3T20_POR.pdf"))

for pos, pagina in enumerate(arquivo_PDF.pages, 1):
    texto_pagina = pagina.extract_text()
    if texto_referencia in texto_pagina:
        texto_analisar = texto_pagina

inicio_texto = texto_analisar.find(texto_referencia)
final_texto = texto_analisar.find("|", inicio_texto + 1)
texto_final = texto_analisar[inicio_texto:final_texto].replace("  ", "")
print(texto_final)




| Despesas com Vendas
 
No 3T20, as despesas com vendas totalizaram R$1.432,6 milhões, equivalentes a 17,2% da receita líquida, 1,1 p.p. menor que no 
3T19 , principalmente devido ao forte crescimento das vendas . Vale ressaltar que a Companhia conseguiu diluir as despesas com 
vendas m esmo investi ndo em maior nível de serviço,especialmente ematendimento e logística.
 
Nos 9M20, as despesas com vendas totalizaram R$3.487,2 milhões, equivalentes a 18,2% da receita líquida (+1,1 p.p. versusos 
9M19).
 



#### 2º Objetivo: Quero analisar o DRE (sem ajuste - Página 5)
    - Para ler tabelas em pdf, use o tabula (é ninja)
    
    - Cuidado 1: Instale o tabula-py (não instale o tabula). Se instalar o tabula errado, desinstale ele, instale o tabula-py, desinstale o tabula-py e instale novamente o tabula-py. Reinicie o kernel do Jupyter após isso
    
    - Cuidado 2: Tem que ter o java instalado no seu computador (depois de instalar, reinicie o computador)

In [26]:
from pathlib import Path
import PyPDF2 as pyf
import tabula
import pandas as pd


CAMINHO_REFERENCIA = Path('./Python txt e PDF 10 - Dados')
tabela = tabula.read_pdf(CAMINHO_REFERENCIA.joinpath(f"MGLU_ER_3T20_POR.pdf"), pages=5, encoding='ANSI')
tabela_limpa = tabela[0]
# Remover linahs vazias
tabela_limpa = tabela_limpa.dropna(how="all", axis=0)
# Remover colunas vazias
tabela_limpa = tabela_limpa.dropna(how="all", axis=1)
tabela_limpa.columns = tabela_limpa.iloc[0]
tabela_limpa = tabela_limpa.iloc[1:]
tabela_limpa = tabela_limpa.reset_index(drop=True)
display(tabela_limpa)

,R$ milhões (exceto quando indicado),3T2,3T1,Var(%,9M2,9M1,Var(%
0,Vendas Totais1 (incluindo marketplace),"12.355,5","6.817,6","81,2","28.584,","18.282,6","56,3"
1,Receita Bruta,"10.349,5","5.999,4","72,5","23.652,","16.508,8","43,3"
2,Receita Líquida,"8.308,3","4.864,2","70,8","19.111,","13.501,3","41,6"
3,Lucro Bruto,"2.178,7","1.424,9","52,9","5.034,","3.728,6","35,0"
4,Margem Bruta,"26,2","29,3","-3,1 p","26,3","27,6","-1,3 p"
5,EBITDA,"546,1","501,2","9,0","1.022,","1.276,5","-19,9"
6,Margem EBITDA,"6,6","10,3","-3,7 p","5,3","9,5","-4,2 p"
7,Lucro Líquido,"206,0","235,1","-12,4","172,","753,8","-77,2"
8,Margem Líquida,"2,5","4,8","-2,3 p","0,9","5,6","-4,7 p"
9,Lucro Bruto - Ajustado,"2.178,7","1.488,9","46,3","5.034,","3.964,6","27,0"


#### 3º Objetivo: Quero analisar o Capital de Giro e os Investimentos (ambas as tabelas na página 12)
    - Páginas com mais de 1 tabela

In [6]:
from pathlib import Path
import tabula


CAMINHO_REFERENCIA = Path('./Python txt e PDF 10 - Dados')
tabelas = tabula.read_pdf(CAMINHO_REFERENCIA.joinpath(f"MGLU_ER_3T20_POR.pdf"), pages=12, encoding='ANSI')
for tabela in tabelas:
    tabela_limpa = tabela
    # Remover linahs vazias
    tabela_limpa = tabela_limpa.dropna(how="all", axis=0)
    # Remover colunas vazias
    tabela_limpa = tabela_limpa.dropna(how="all", axis=1)
    tabela_limpa = tabela_limpa.reset_index(drop=True)
    display(tabela_limpa)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,(+) Contas a Receber (sem Cartões de Crédito),"(26,7)","706,3","680,8","781,3","794,0","733,0"
1,(+) Estoques,"2.120,2","5.005,9","4.198,2","4.075,5","3.801,8","2.885,7"
2,(+) Partes Relacionadas (sem Cartão Luiza),"(10,5)","71,3","80,4","77,1","100,6","81,8"
3,(+) Impostos a Recuperar,"186,3","932,0","748,9","877,4","864,1","745,7"
4,(+) Outros Ativos,"(56,6)","88,5","100,2","143,5","136,3","145,1"
5,(+) Ativos Circulantes Operacionais,"2.212,7","6.804,0","5.808,6","5.954,8","5.696,8","4.591,3"
6,(-) Fornecedores,"2.301,5","6.104,3","5.334,0","4.132,7","5.934,9","3.802,8"
7,(-) Repasses e outros depósitos,"627,3","627,3","639,3","235,9",-,-
8,"(-) Salários, Férias e Encargos Sociais","95,0","444,7","329,0","263,3","354,7","349,8"
9,(-) Impostos a Recolher,"90,7","299,6","206,4","176,9","352,0","208,8"


,R$ milhões,3T20,%,3T19,%.1,Var(%),9M20,%.2,9M20.1,%.3,Var(%).1
0,Lojas Novas,"21,2",14%,"94,6",51%,-78%,"69,0",21%,"121,9",31%,-43%
1,Reformas,"6,1",4%,"8,2",4%,-26%,"14,6",4%,"38,2",10%,-62%
2,Tecnologia,"69,1",45%,"32,6",18%,112%,"147,6",45%,"84,4",22%,75%
3,Logística,"36,3",24%,"32,8",18%,11%,"62,1",19%,"107,4",27%,-42%
4,Outros,"21,5",14%,"17,7",10%,22%,"35,7",11%,"38,8",10%,-8%
5,Total,"154,2",100%,"186,0",100%,-17%,"329,1",100%,"390,7",100%,-16%


#### O que fazer quando o tabula não consegue ler alguma linha da tabela? Como o cabeçalho, no nosso caso?

In [16]:
df_capital_giro = tabelas[0]
# Remover linahs vazias
df_capital_giro = df_capital_giro.dropna(how="all", axis=0)
# Remover colunas vazias
df_capital_giro = df_capital_giro.dropna(how="all", axis=1)
df_capital_giro = df_capital_giro.reset_index(drop=True)
# display(df_capital_giro)
tabelas2 = tabula.read_pdf(CAMINHO_REFERENCIA.joinpath(f"MGLU_ER_3T20_POR.pdf"), pages=12, encoding='ANSI', lattice=True)
df_capital_giro2 = tabelas2[0]
df_capital_giro2 = df_capital_giro2.dropna(how="all", axis=0)
df_capital_giro2 = df_capital_giro2.dropna(how="all", axis=1)
df_capital_giro2 = df_capital_giro2.reset_index(drop=True)
colunas = df_capital_giro2.iloc[0]
colunas = colunas.dropna()
df_capital_giro.columns = colunas
display(df_capital_giro)

,R$ milhões,Dif 12U,set-2,jun-2,mar-2,dez-1,set-1
0,(+) Contas a Receber (sem Cartões de Crédito),"(26,7)","706,3","680,8","781,3","794,0","733,0"
1,(+) Estoques,"2.120,2","5.005,9","4.198,2","4.075,5","3.801,8","2.885,7"
2,(+) Partes Relacionadas (sem Cartão Luiza),"(10,5)","71,3","80,4","77,1","100,6","81,8"
3,(+) Impostos a Recuperar,"186,3","932,0","748,9","877,4","864,1","745,7"
4,(+) Outros Ativos,"(56,6)","88,5","100,2","143,5","136,3","145,1"
5,(+) Ativos Circulantes Operacionais,"2.212,7","6.804,0","5.808,6","5.954,8","5.696,8","4.591,3"
6,(-) Fornecedores,"2.301,5","6.104,3","5.334,0","4.132,7","5.934,9","3.802,8"
7,(-) Repasses e outros depósitos,"627,3","627,3","639,3","235,9",-,-
8,"(-) Salários, Férias e Encargos Sociais","95,0","444,7","329,0","263,3","354,7","349,8"
9,(-) Impostos a Recolher,"90,7","299,6","206,4","176,9","352,0","208,8"


# Outro método que pode ser útil algum dia: Captar Imagem em um pdf
    - biblioteca pikepdf

# Substituir texto no pdf tipo contrato

- Não recomendo fazer diretamente pelo Python. Realmente do que vi a melhor opção me parece o Word fazer isso
- Caso precise automatizar, automatize o processo fazendo ele pelo Word
- Quem quiser MUITO fazer isso pelo Python, tem um link aqui que vai te ajudar de uma solução que achei que funciona. Tem seus bugs/cuidados especiais, mas funciona: https://pdf.co/samples/pdf-co-web-api-replace-text-from-pdf-python-replace-text-from-url